In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/Kaggle\ Competition/amex

/content/gdrive/MyDrive/Kaggle Competition/amex


In [ ]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
# import lightgbm as lgb
from itertools import combinations

In [ ]:
class CFG:
    input_dir = 'data/'
    seed = 42
    n_folds = 5
    target = 'target'

In [ ]:
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

In [ ]:
# #load lgbm
# lgbm_oof = pd.read_csv('Jason_models/LGBM/OOF/oof_lgbm_baseline_5fold_seed42.csv')
# lgbm_pred = pd.read_csv('Jason_models/LGBM/Predictions/test_lgbm_baseline_5fold_seed42.csv')
# #load xgb
# xgb_oof = pd.read_csv('Jason_models/XGB/OOF/oof_xgb_baseline_5fold_seed42.csv')
# xgb_pred = pd.read_csv('Jason_models/XGB/Predictions/test_xgb_baseline_5fold_seed42.csv')
# #load catb
# catb_oof = pd.read_csv('Jason_models/CatB/OOF/oof_catb_baseline_5fold_seed42.csv')
# catb_pred = pd.read_csv('Jason_models/CatB/Predictions/test_catb_baseline_5fold_seed42.csv')

In [ ]:
#load lgbm
lgbm_oof = pd.read_csv('2nd_round/Jason_models/LGBM/OOF/oof_lgbm_dart_baseline_5fold_seed42.csv')
lgbm_pred = pd.read_csv('2nd_round/Jason_models/LGBM/Predictions/test_lgbm_dart_baseline_5fold_seed42.csv')
#load xgb
xgb_oof = pd.read_csv('models/xgb_fe_3/oof_pred.csv')
xgb_pred = pd.read_csv('submissions/xgb_submission_fe_3.csv')
#load catb
catb_oof = pd.read_csv('models/cat_fe_3/oof_pred.csv')
catb_pred = pd.read_csv('submissions/cat_submission_fe_3.csv')

In [ ]:
def read_data():
    train = pd.read_parquet(CFG.input_dir + 'train_fe.parquet')
    test = pd.read_parquet(CFG.input_dir + 'test_fe.parquet')
    return train, test

In [ ]:
train, test = read_data()

In [ ]:
print(train[CFG.target].shape)

(458913,)


In [ ]:
amex_metric(train[CFG.target], lgbm_oof['prediction'])

0.797071726239252

In [ ]:
xgb_oof

In [ ]:
amex_metric(train[CFG.target], xgb_oof['Predictions'])

0.7938798685033689

In [ ]:
amex_metric(train[CFG.target], catb_oof['Predictions'])

0.7952659162397979

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def prob_2_log(p):
  return np.log(p/(1-p))

def log_2_prob(log):
  return np.exp(log)/(1 + np.exp(log))

In [ ]:
comb = []
score = []
for i in tqdm(range(0, 100, 1)):
  for j in range(0, 100 - i, 1):
    i /= 100
    j /= 100
    k = 1.0 - i - j
    comb.append((i, j, k))
    # print(i, j, k)
    log_sum = i*prob_2_log(lgbm_oof['prediction']) + j*prob_2_log(xgb_oof['Predictions']) + k*prob_2_log(catb_oof['Predictions'])

    ensemble = log_2_prob(log_sum)
    # print(np.shape(train[CFG.target]))
    # print(np.shape(ensemble))
    val = amex_metric(train[CFG.target], ensemble)
    score.append(val)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
for a, b, c in zip(lgbm_oof['prediction'], xgb_oof['Predictions'], catb_oof['Predictions']):
    print(max(a, b, c) - min(a, b, c))

In [ ]:
np.argmax(score)

4230

In [ ]:
comb[4230]

(0.6, 0.0, 0.4)

In [ ]:
score[4230]

0.7978616765045837

In [ ]:
comb[4015]

(0.55, 0.0, 0.44999999999999996)

In [ ]:
score[4015]

0.7979093639000079

In [ ]:
amex_metric(train[CFG.target], lgbm_oof['prediction'])

0.797071726239252

In [ ]:
result = lgbm_pred.copy()
result

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.029931
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000848
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.031462
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.163582
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.872624
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.012778
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.819399
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.431248
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.243115


In [ ]:
# result['prediction'] =0.75*lgbm_pred['prediction'] + 0.05*xgb_pred['prediction'] + 0.20*catb_pred['prediction']

In [ ]:
result

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.028597
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.001013
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.031488
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.168138
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.857276
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.012705
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.814006
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.442444
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.255858


In [ ]:
def prob_2_log(p):
  return np.log(p/(1-p))

def log_2_prob(log):
  return np.exp(log)/(1 + np.exp(log))

In [ ]:
log_sum = 0.6*prob_2_log(lgbm_pred['prediction']) + 0.10*prob_2_log(xgb_pred['prediction']) + 0.30*prob_2_log(catb_pred['prediction'])

result['prediction'] = log_2_prob(log_sum)

result

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.028137
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.001058
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.031747
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.172713
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.863584
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.012866
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.824490
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.457628
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.265630


In [ ]:
amex_metric(train[CFG.target], result['prediction'])

IndexError: ignored

In [ ]:
result.to_csv('Jul31_submission.csv', index = False)